In [1]:
import pandas as pd
import numpy as np

# Define the special Hugging Face URL for the CSV file
# This tells pandas to use fsspec and huggingface_hub to find and read the file
hf_csv_url = "hf://datasets/MatrixIA/FraudData/FraudData.csv"

print(f"Attempting to load full dataset from: {hf_csv_url}")
try:
    # Use pandas read_csv directly with the hf:// URL
    # This loads the entire dataset into memory. May take a minute or two.
    df_full = pd.read_csv(hf_csv_url)
    print("Successfully loaded full dataset into Pandas DataFrame.")
    print(f"Full DataFrame shape: {df_full.shape}")

    # --- IMPORTANT: Create a Sample for Development ---
    # Define sample size (e.g., 1 million rows)
    sample_size = 1000000
    print(f"Creating a sample of {sample_size} rows for initial development...")

    # Option 1: Take the first N rows (simplest)
    df = df_full.head(sample_size).copy()

    # Option 2: Take a random sample (better representation, might be slightly slower)
    # df = df_full.sample(n=sample_size, random_state=42).copy()

    print(f"Sample DataFrame shape: {df.shape}")

    # Optional: Delete the full dataframe to free memory if you notice slowdowns
    # Although with 24GB RAM, it might not be necessary yet.
    # del df_full
    # import gc # Garbage collector
    # gc.collect() # Force memory cleanup

except Exception as e:
    # Catch potential errors during download or reading
    print(f"ERROR: Failed to load dataset using pd.read_csv('hf://...'). Error: {e}")
    print("Check your internet connection, proxy settings (if any), and the URL.")
    raise # Stop execution

# --- Basic Inspection (Run on the SAMPLE 'df') ---
# This part remains the same as before, using the 'df' variable which now holds the sample
print("\nFirst 5 rows (from sample):")
print(df.head())

print("\nDataFrame Info (from sample):")
df.info()

print("\nSummary Statistics (from sample):")
print(df.describe())

print("\nMissing Values Count per Column (from sample):")
print(df.isnull().sum())

# --- Target Variable Check (Run on the SAMPLE 'df') ---
# Target column is 'isFraud' for this dataset (PaySim)
target_column = 'isFraud'

if target_column in df.columns:
    print(f"\nClass Distribution for Target Column ('{target_column}') in Sample:")
    print(df[target_column].value_counts(normalize=True))
    print("\nRaw Counts in Sample:")
    print(df[target_column].value_counts(normalize=False))
else:
    print(f"\nERROR: Target column '{target_column}' not found in the DataFrame!")
    print(f"Available columns are: {list(df.columns)}")

Attempting to load full dataset from: hf://datasets/MatrixIA/FraudData/FraudData.csv
Successfully loaded full dataset into Pandas DataFrame.
Full DataFrame shape: (6362620, 11)
Creating a sample of 1000000 rows for initial development...
Sample DataFrame shape: (1000000, 11)

First 5 rows (from sample):
   step      type    amount     nameOrig  oldbalanceOrg  newbalanceOrig  \
0     1   PAYMENT   9839.64  C1231006815       170136.0       160296.36   
1     1   PAYMENT   1864.28  C1666544295        21249.0        19384.72   
2     1  TRANSFER    181.00  C1305486145          181.0            0.00   
3     1  CASH_OUT    181.00   C840083671          181.0            0.00   
4     1   PAYMENT  11668.14  C2048537720        41554.0        29885.86   

      nameDest  oldbalanceDest  newbalanceDest  isFraud  isFlaggedFraud  
0  M1979787155             0.0             0.0        0               0  
1  M2044282225             0.0             0.0        0               0  
2   C553264065        

In [2]:
# Import the profiling tool
try:
    from ydata_profiling import ProfileReport
except ImportError:
    try:
        from pandas_profiling import ProfileReport
    except ImportError:
        print("ERROR: Neither ydata-profiling nor pandas-profiling seem to be installed.")
        print("Please run: pip install ydata-profiling")
        raise # Stop if library isn't installed

print("\nGenerating data profiling report on the SAMPLE... (This might take a minute or two)")

# Create the report object using the SAMPLE DataFrame 'df'
profile = ProfileReport(df, title="Fraud Data Profiling Report (Sample)", explorative=True)

# Define the filename for the HTML report
report_filename = "fraud_data_profiling_report_sample.html"

# Save the report to an HTML file
profile.to_file(report_filename)

print(f"Profiling report saved to {report_filename}")


Generating data profiling report on the SAMPLE... (This might take a minute or two)


Summarize dataset:   0%|          | 0/5 [00:00<?, ?it/s]


100%|████████████████████████████████████████████████| 11/11 [00:09<00:00,  1.12it/s]


Generate report structure:   0%|          | 0/1 [00:00<?, ?it/s]

Render HTML:   0%|          | 0/1 [00:00<?, ?it/s]

Export report to file:   0%|          | 0/1 [00:00<?, ?it/s]

Profiling report saved to fraud_data_profiling_report_sample.html
